In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# import sys
# os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

In [2]:
from models.unet import Unet

In [3]:
input_shape = (512,512,3)

In [ ]:
model = Unet(input_shape, feature=16, levels=4)  

In [15]:
model.summary()

Model: "Unet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
unet_layer_5 (UnetLayer)     (None, 512, 512, 32)      5487712   
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 512, 512, 3)       99        
Total params: 5,487,811
Trainable params: 5,487,811
Non-trainable params: 0
_________________________________________________________________
